In [1]:
import pandas as pd
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


Matplotlib is building the font cache; this may take a moment.


In [2]:
parquet_file_path = '/Users/sid/Desktop/Personel/SJSU/PP CP Research/Data/training_lidar_10017090168044687777_6380_000_6400_000.parquet'
df = pd.read_parquet(parquet_file_path)


In [8]:
df.columns

Index(['key.segment_context_name', 'key.frame_timestamp_micros',
       'key.laser_name', '[LiDARComponent].range_image_return1.values',
       '[LiDARComponent].range_image_return1.shape',
       '[LiDARComponent].range_image_return2.values',
       '[LiDARComponent].range_image_return2.shape'],
      dtype='object')

In [9]:
df.columns

Index(['key.segment_context_name', 'key.frame_timestamp_micros',
       'key.laser_name', '[LiDARComponent].range_image_return1.values',
       '[LiDARComponent].range_image_return1.shape',
       '[LiDARComponent].range_image_return2.values',
       '[LiDARComponent].range_image_return2.shape'],
      dtype='object')

In [12]:
range_image_return1_values = df['[LiDARComponent].range_image_return1.values']
range_image_return1_shape = df['[LiDARComponent].range_image_return1.shape']

In [15]:
range_image_return1_values.head()

index
10017090168044687777_6380_000_6400_000;1550083467346370    [20.885931, 0.19042969, 1.493107, -1.0, 18.936...
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
Name: [LiDARComponent].range_image_return1.values, dtype: object

In [13]:
range_image_return1_shape

index
10017090168044687777_6380_000_6400_000;1550083467346370    [64, 2650, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
                                                               ...      
10017090168044687777_6380_000_6400_000;1550083487048517    [64, 2650, 4]
10017090168044687777_6380_000_6400_000;1550083487048517    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083487048517    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083487048517    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083487048517    [200, 600, 4]
Name: [LiDARComponent].range_image_return1.shape, Length: 990, dtype: object

In [16]:
range_image_return2_values = df['[LiDARComponent].range_image_return2.values']
range_image_return2_shape = df['[LiDARComponent].range_image_return2.shape']

In [19]:
range_image_return2_values.head()

index
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, 21.003038, 0.14941406...
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
10017090168044687777_6380_000_6400_000;1550083467346370    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
Name: [LiDARComponent].range_image_return2.values, dtype: object

In [20]:
range_image_return2_shape.head()

index
10017090168044687777_6380_000_6400_000;1550083467346370    [64, 2650, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
10017090168044687777_6380_000_6400_000;1550083467346370    [200, 600, 4]
Name: [LiDARComponent].range_image_return2.shape, dtype: object

In [21]:
def create_range_point_cloud(value, shape):
  
  range_image = np.array(value)
  range_image_reshaped = range_image.reshape(shape)

  
  ranges = range_image_reshaped[..., 0]
  azimuths = range_image_reshaped[..., 1]
  elevations = range_image_reshaped[..., 2]

  
  x = ranges * np.cos(elevations) * np.sin(azimuths)
  y = ranges * np.cos(elevations) * np.cos(azimuths)
  z = ranges * np.sin(elevations)

  
  points = np.stack((x, y, z), axis=-1).reshape(-1, 3)

  
  point_cloud = o3d.geometry.PointCloud()
  point_cloud.points = o3d.utility.Vector3dVector(points)

  
  o3d.visualization.draw_geometries([point_cloud])

In [23]:
value = range_image_return1_values.iloc[0]
shape = range_image_return1_shape.iloc[0]
create_range_point_cloud(value, shape)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 

In [ ]:
points = df[['x', 'y', 'z']].to_numpy()


point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)


o3d.visualization.draw_geometries([point_cloud])